# Data Preprocessing for SuStaIn analysis

In [ ]:
import pandas as pd
import numpy as np
import neuroCombat
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.linalg import svd
import os, sys

This notebook lays out data cleaning and preprocessing steps to investigate the disease progression based subtyping [(SuStaIn)](https://www.nature.com/articles/s41467-018-05892-0) of patients with epilepsy using the [ENIGMA](https://enigma.ini.usc.edu/) dataset. Please note that the results have been withheld to safeguard data protection.

In [ ]:
#Some measures acquired from T1W MRI contains missing values. In order to impute these values, we first use  
#SVD (Singular Value Decomposition) which factorises the input matrix as a product of U, S and V matrices. 
#The second step is to regenerate the input matix from the dot product of U*S*V. 
#In the third step we simply substitute the newly generated values in place of the values missing from the original
#matrix.

#Here, input is data matrix M, and SVD components k to use for reconstructions, epsilon is the minimal
#improvment between interations (may be data dependent)
#NOTE: if k is too large we may reintroduce noise, best: screen for good k by generating missing data
def SVDimpute(M, k=5, eps=0.001):
    #center matrix
    cmeans = np.nanmean(M, axis=0)
    csds = np.nanstd(M, axis=0)
    #csds = np.ones(cmeans.shape)
    M1 = (M - cmeans)/csds

    #replace missing data with 0 (i.e., mean imputation)
    M2 = np.copy(M1)
    missing = np.isnan(M1)
    M2[missing] = 0.0
    
    diff = 100000000
    while diff > eps:
        U, s, Vh = svd(M2)
        sigma = np.zeros(M.shape)
        for i in range(min(M.shape)):
            sigma[i, i] = s[i]        
        recon = np.dot(U[:,0:k], np.dot(sigma[0:k,0:k], Vh[0:k,:]))
        diff = np.sum((M2[missing] - recon[missing])**2)
        #print(diff)
        M2[missing] = recon[missing]
    res = np.copy(M2) * csds + cmeans
    return res

In [ ]:
def optSVDimpute(M, eps=0.01, ks=np.arange(2,20)):
    #generate 5% missing data
    miss = np.random.rand(M.shape[0], M.shape[1]) > 0.95
    tmp = np.copy(M)
    tmp[miss] = np.nan
    
    best_k = 0
    best_diff = 100000000
    for k in ks:
        tmp_imp = SVDimpute(tmp, k, eps)
        ttt = (tmp_imp[miss] - M[miss])**2
        ttt[np.isnan(ttt)] = 0
        diff=np.sqrt(sum(ttt))
        if diff < best_diff:
            best_k = k
            best_diff = diff
            print(best_k)
            print(best_diff)
    return SVDimpute(M, best_k, eps)

In [ ]:
#That is simply area \times avgthick
def generateVolFeature(dat, rois, lat='L'):
    res = pd.DataFrame()
    fnames = []
    for roi in rois:
        tmp = dat.loc[:, lat + "_" + roi + "_surfavg"] * dat.loc[:, lat + "_" + roi + "_thickavg"]
        fnames.append(lat + "_" + roi + "_volume")
        res = pd.concat([res, tmp], axis=1)
    res.index = dat.index
    res.columns = fnames
    return(res)


In [ ]:
def generateLobeFeature(dat, assign, lat='L', feat='surfavg'):
    res = pd.DataFrame()
    fnames=[]
    for lobe in assign:
        roilist = assign[lobe]
        tmp_feat = []
        for roi in roilist:
            tmp_feat.append(lat + "_" + roi + "_" + feat)
        tmp = dat.loc[:,tmp_feat].apply(lambda x: np.sum(x), axis=1)
        fnames.append(lat + "_" + lobe + "_" + feat)
        res = pd.concat([res, tmp], axis=1)
    res.index = dat.index
    res.columns=fnames
    return(res)


In [ ]:
def cohenD(y, idx):
    mdiff = y[idx].mean() - y[~idx].mean()
    return(mdiff/y.std())

def robustCohenD(y, idx):
    mdiff = y[idx].median() - y[~idx].median()
    m1 = y.median()
    mad = np.median(np.abs(y - m1))
    return (mdiff/ (1.4826 * mad ) )

In [ ]:
from sklearn import linear_model

#using a linear model
def regressOut(y, X, use_fit=None):
    lm = linear_model.LinearRegression()    
    if use_fit is None:
        use_fit = [True] * X.shape[0]
    else:
        use_fit = use_fit.values

    x_mean = X.iloc[use_fit,:].mean()
    lm.fit(X.loc[use_fit,:].values, y.loc[use_fit])
    yhat = lm.predict(X.values)
    #residual
    res = y - yhat
    #add intercept and average of X
    #offset = lm.intercept_ + lm.predict(x_mean.values.reshape(1, -1))
    offset = lm.predict(x_mean.values.reshape(1, -1))
    #toadd = lm.intercept_ + lm.coef_ * X.mean()
    #return(res + toadd)
    return(res + offset)

## Prepare T1W data

In [ ]:
#load t1 data
t1_dat = pd.read_csv(mypath + "T1_data_all_20220114.csv", index_col=0)
t1_dat.head()

In [ ]:
# fix ICV
t1_dat.loc[:,"ICV":"ICV.3"] = t1_dat.loc[:,"ICV":"ICV.3"].apply(lambda x: pd.to_numeric(x, errors='coerce'), axis=0)
t1_dat.ICV = t1_dat.loc[:,"ICV.3"]
t1_dat.loc[np.isnan(t1_dat.ICV),"ICV"] = np.nanmedian(t1_dat.ICV)

In [ ]:
#remove t1s with too much missing data
#t1_dat.loc[:, "LLatVent":"R_insula_surfavg"].apply(lambda x: sum(np.isnan(x)), axis=1).value_counts()
#let's take 20 as a cutoff based on the above
keep_idx = t1_dat.loc[:, "LLatVent":"R_insula_surfavg"].apply(lambda x: sum(np.isnan(x)), axis=1) < 20

In [ ]:
t1_dat = t1_dat[keep_idx]

In [ ]:
subj_info = t1_dat.loc[:,'Site':'ICV']

In [ ]:
kidx2 = ~subj_info.loc[:,'ICV'].isna() & ~subj_info.loc[:,'Age'].isna() & (subj_info.loc[:,'Age'] > 0.0) & (subj_info.Site != 'Genova')
t1_dat = t1_dat[kidx2]


In [ ]:
# split into types of features
subj_info = t1_dat.loc[:,'Site':'ICV']
subj_feat_sv  = t1_dat.loc[:,'LLatVent':'Raccumb']
subj_feat_ct  = t1_dat.loc[:,'L_bankssts_thickavg':'R_insula_thickavg']
subj_feat_sa  = t1_dat.loc[:,'L_bankssts_surfavg':'R_insula_surfavg']

In [ ]:
subj_feat = pd.concat([ subj_feat_sv, subj_feat_ct, subj_feat_sa ], axis=1)
subj_feat.index = subj_info.index
subj_feat.shape

### impute missing t1w data

In [ ]:
centers = set(subj_info.Site)

In [ ]:
subj_features_imputed = pd.DataFrame()
subj_info_imp = pd.DataFrame()

for site in centers:
    print(site)
    sidx = subj_info.Site == site
    print(sum(sidx))
    tmp  = pd.DataFrame(optSVDimpute(subj_feat[sidx].values))
    subj_features_imputed = subj_features_imputed.append(tmp)
    subj_info_imp = subj_info_imp.append(subj_info[sidx])


In [ ]:
#add index and column names!
subj_features_imputed.columns= subj_feat.columns
subj_features_imputed.index  = subj_info_imp.index
subj_info_imp.SDx.value_counts()

### generate cortical volume features

In [ ]:
myrois = [roi.replace("L_","").replace("_thickavg","") for roi in t1_dat.loc[:,'L_bankssts_thickavg':'L_insula_thickavg'].columns]


In [ ]:
left_vol  = generateVolFeature(subj_features_imputed, myrois, "L")
right_vol = generateVolFeature(subj_features_imputed, myrois, "R")

subj_features_imputed_addvol = pd.concat([subj_features_imputed, left_vol, right_vol], axis=1)

### generate lobe features

In [ ]:
#Combine ROIs into lobes as defined in https://surfer.nmr.mgh.harvard.edu/fswiki/CorticalParcellation
lobe_assign = {}
lobe_assign['frontalL']  = ['superiorfrontal','rostralmiddlefrontal','caudalmiddlefrontal',
                          'parsopercularis','parstriangularis','parsorbitalis',
                         'lateralorbitofrontal','medialorbitofrontal','precentral',
                         'paracentral','frontalpole']
lobe_assign['parietalL'] = ['superiorparietal','inferiorparietal','supramarginal',
                          'postcentral','precuneus']
lobe_assign['temporalL'] = ['superiortemporal', 'middletemporal', 'inferiortemporal',
                          'bankssts', 'fusiform', 'transversetemporal',
                          'entorhinal', 'temporalpole', 'parahippocampal']
lobe_assign['occipitalL']= ['lateraloccipital', 'lingual', 'cuneus', 'pericalcarine']
lobe_assign['cingulateC']= ['rostralanteriorcingulate', 'caudalanteriorcingulate','posteriorcingulate',
                          'isthmuscingulate']


In [ ]:
left_vo  = generateLobeFeature(subj_features_imputed_addvol, lobe_assign, 'L', feat='volume')
right_vo = generateLobeFeature(subj_features_imputed_addvol, lobe_assign, 'R', feat='volume')


In [ ]:
subj_features_imputed_lobes = pd.concat([subj_features_imputed_addvol, left_vo, right_vo], axis=1)


In [ ]:
#for this analysis we only want to keep subcortical volumes and lobe volumes
keep_feat = list(subj_feat_sv.columns)
keep_feat.extend(left_vo.columns)
keep_feat.extend(right_vo.columns)

In [ ]:
t1_dat_use = pd.concat( [subj_info_imp, subj_features_imputed_lobes.loc[:,keep_feat]], axis=1 )

## Prepare DTI FA data

In [ ]:
#load FA data
fa_dat = pd.read_csv(mypath + 'FA_matched.csv')
fa_dat.head()

In [ ]:
fa_dat.columns.values

In [ ]:
keep_dti_bi_feats  = ['CST','EC','FX.ST','PTR','SLF','SS']
keep_dti_agg_feats = ['IC','CR', 'FO', 'CG']
keep_dti_feats     = ['CC']

In [ ]:
# group DTI features
dtigrp = {}
#dtigrp['IC_manual'] = ['ALIC','PLIC','RLIC']
#dtigrp['CR_manual'] = ['ACR','SCR','PCR']
#dtigrp['CC_manual'] = ['GCC','BCC','SCC']
dtigrp['CG'] = ['CGC','CGH']
dtigrp['FO'] = ['IFO','SFO']


In [ ]:
def aggregateROIs(df, roilist, lat="L"):
    rois = [ ".".join( [x, lat]) for x in roilist]
    tmp = df.loc[:, rois]
    return (tmp.apply(np.mean, axis=1))

In [ ]:
manual_dti = {}
manual_dti['CG.L'] = aggregateROIs(fa_dat, dtigrp['CG'], "L")
manual_dti['FO.L'] = aggregateROIs(fa_dat, dtigrp['FO'], "L")
manual_dti['CG.R'] = aggregateROIs(fa_dat, dtigrp['CG'], "R")
manual_dti['FO.R'] = aggregateROIs(fa_dat, dtigrp['FO'], "R")

In [ ]:
manual_dti = pd.DataFrame(manual_dti)
manual_dti.index = fa_dat.index

In [ ]:
fa_dat2 = pd.concat([fa_dat, manual_dti], axis=1)

In [ ]:
keep_cols = ['Site','SubjID', 'CC']
for x in keep_dti_bi_feats:
    keep_cols.append(x + ".L")
    keep_cols.append(x + ".R")
for x in keep_dti_agg_feats:
    keep_cols.append(x + ".L")
    keep_cols.append(x + ".R")    

In [ ]:
fa_dat3 = fa_dat2.loc[:, keep_cols]

In [ ]:
fa_dat3.columns.values

## Prepare MD data

In [ ]:
#load MD data
md_dat = pd.read_csv(mypath + 'MD_matched.csv')
md_dat.head()

In [ ]:
#Exclude sites with outliers
md_dat = md_dat[(md_dat.Site != 'EKUT') & (md_dat.Site != 'UMG')]

In [ ]:
#remove subject 'EPICZ', 'subj_CON_100' who has a outlier value
rm_sub = (md_dat.Site == 'EPICZ') & (md_dat.SubjID=='subj_CON_100')
md_dat = md_dat[~rm_sub]

In [ ]:
#deal with aggregate features
manual_md = {}
manual_md['CG.L'] = aggregateROIs(md_dat, dtigrp['CG'], "L")
manual_md['FO.L'] = aggregateROIs(md_dat, dtigrp['FO'], "L")
manual_md['CG.R'] = aggregateROIs(md_dat, dtigrp['CG'], "R")
manual_md['FO.R'] = aggregateROIs(md_dat, dtigrp['FO'], "R")

manual_md = pd.DataFrame(manual_md)
manual_md.index = md_dat.index

In [ ]:
md_dat2 = pd.concat([md_dat, manual_md], axis=1)

In [ ]:
md_dat3 = md_dat2.loc[:, keep_cols]

In [ ]:
#merge DTI features
dti_dat = fa_dat3.merge(md_dat3, on=['Site','SubjID'], suffixes=['_FA','_MD'])
dti_dat.head()

In [ ]:
#merge with T1 data
img_dat = dti_dat.merge(t1_dat_use, on=['Site','SubjID'], suffixes=['','_T1'])
img_dat.head()

In [ ]:
#add DTI covariates
dti_cov = pd.read_csv(mypath + "cov_dti_matched.csv")
dti_cov.SDx = pd.to_numeric(dti_cov.SDx, errors='coerce')
#dti_cov.head()
#img_dat_bak = img_dat.copy()
img_datx = img_dat.merge(dti_cov, left_on=["Site","SubjID"], right_on=["Site","SubjID"], suffixes=('', '_dti'))

In [ ]:
img_dat = img_datx.copy()

In [ ]:
#remove sujects with missing SDx_dti
img_dat = img_dat[~np.isnan(img_dat.SDx_dti)]

In [ ]:
print(fa_dat3.shape)
print(md_dat3.shape)
print(t1_dat_use.shape)
print(img_dat.shape)


In [ ]:
fa_dat3.describe()

In [ ]:
md_dat3.describe()

In [ ]:
def ridgePlotQuick(fname, my_df):
    feat = fname
    sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0), 'axes.linewidth':2})

    palette = sns.color_palette("Set2", 12)

    g = sns.FacetGrid(my_df, row="Site", hue="Site", aspect=9, height=1.2)

    g.map_dataframe(sns.kdeplot, x=feat, fill=True, alpha=0.8)
    g.map_dataframe(sns.kdeplot, x=feat, color='black')

    def label(x, color, label):
        ax = plt.gca()
        ax.text(0, .2, label, color='black', fontsize=13,
            ha="left", va="center", transform=ax.transAxes)
    
    g.map(label, "Site")

    g.fig.subplots_adjust(hspace=-.5)
    g.set_titles("")
    g.set(yticks=[])
    g.despine(left=True)

#g.set_titles("")
#g.set(yticks=[], ylabel="", xlabel=nicerNames(feat) )
#g.despine( left=True)


In [ ]:
ridgePlotQuick('CC', fa_dat3)

In [ ]:
ridgePlotQuick('CC', md_dat3)

In [ ]:
tmp = img_dat.describe()

In [ ]:
img_dat.columns.values

In [ ]:
img_dat.SDx_dti.value_counts()

In [ ]:
#normalize FA
fa = img_dat.loc[:,'CC_FA':'CG.R_FA']
nc_fa = neuroCombat.neuroCombat(dat=fa.T, covars=img_dat, batch_col='Site', categorical_cols=['SDx_dti','Sex'], continuous_cols=['Age','ICV'] )


In [ ]:
#normalize MD
md = img_dat.loc[:,'CC_MD':'CG.R_MD']
nc_md = neuroCombat.neuroCombat(dat=md.T, covars=img_dat, batch_col='Site', categorical_cols=['SDx_dti','Sex'], continuous_cols=['Age','ICV'] )
#nc_md = neuroCombat.neuroCombat(dat=md.T, covars=img_dat, batch_col='Site', categorical_cols=['Dx','Sex'], continuous_cols=['Age'] )


In [ ]:
#normalize T1
t1w = img_dat.loc[:,'LLatVent':'R_cingulateC_volume']
nc_t1w = neuroCombat.neuroCombat(dat=t1w.T, covars=img_dat, batch_col='Site', categorical_cols=['SDx_dti','Sex'], continuous_cols=['Age','ICV'] )


In [ ]:
img_dat_nc = img_dat.copy()


In [ ]:
img_dat_nc.loc[:,'CC_FA':'CG.R_FA'] = nc_fa['data'].T
img_dat_nc.loc[:,'CC_MD':'CG.R_MD'] = nc_md['data'].T
img_dat_nc.loc[:,'LLatVent':'R_cingulateC_volume'] = nc_t1w['data'].T

In [ ]:
img_dat_nc.loc[:,'CC_FA':'CG.R_FA'].describe()

In [ ]:
img_dat_nc.loc[:,'CC_MD':'CG.R_MD'].describe()

In [ ]:
img_dat_nc.loc[:,'LLatVent':'R_cingulateC_volume'].describe()

In [ ]:
ridgePlotQuick('CC_FA', img_dat_nc)

In [ ]:
ridgePlotQuick('CC_MD', img_dat_nc)

In [ ]:
ridgePlotQuick('Rhippo', img_dat_nc)

In [ ]:
mycols = ['b','r']
mycvec = [ mycols[x] for x in img_dat.Dx.values]
plt.scatter(img_dat_nc.ICV, img_dat_nc.Lhippo, c=mycvec)

# Create normative model wrt Age for controls

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct, WhiteKernel, ConstantKernel as C
from sklearn.preprocessing import StandardScaler

In [ ]:
img_dat_nc_controls = img_dat_nc[(img_dat_nc.Dx==0)]

### Pack the GPR into a few neat functions

In [ ]:
def fitNormGP(mydf, cov, yname):
    use_kernel = C() * RBF() + WhiteKernel(noise_level_bounds=(1e-06, 100000.0))
    myGPR = GaussianProcessRegressor(n_restarts_optimizer=5, kernel=use_kernel, normalize_y=True)
    
    X = mydf.loc[:, cov]
    tmp_scale = StandardScaler()
    Xsc = tmp_scale.fit_transform(X)
    
    y = mydf.loc[:, yname]
    
    myGPR.fit(Xsc, y)
    
    res = {}
    res['target'] = yname
    res['cov'] = cov
    res['scaler'] = tmp_scale
    res['GPR'] = myGPR
    return(res)

In [ ]:
def feat2z(mydf, gpmodel):
    #assert(gpmodel['target'] == yname)
    yname = gpmodel['target']
    cov = gpmodel['cov']
    X = mydf.loc[:,cov]
    y = mydf.loc[:, yname]
    
    Xinp = gpmodel['scaler'].transform(X)
    
    means, sds = gpmodel['GPR'].predict(Xinp, return_std=True)
    
    myz = (y-means)/sds
    
    return(myz)

In [ ]:
def plotGPax(mydf, gpmodel, var=0):

    cov = gpmodel['cov']
    X = mydf.loc[:,cov]
    Xinp = gpmodel['scaler'].transform(X)
    
    nx = 1000
    x1 = np.linspace(np.min(Xinp[:,var]), np.max(Xinp[:,var]), nx)
    xx = np.zeros( (nx, X.shape[1]) )
    xx[:,var] = x1
    mm, ss = gpmodel['GPR'].predict(xx, return_std=True)

    x1p = x1 * np.sqrt(gpmodel['scaler'].var_[var]) + gpmodel['scaler'].mean_[var]
    
    fig, ax = plt.subplots()
    ax.fill_between(x1p, mm-1*ss, mm+1*ss, alpha=0.5, linewidth=0)
    ax.fill_between(x1p, mm-1.96*ss, mm+1.96*ss, alpha=0.5, linewidth=0)
    ax.plot(x1p, mm, linewidth=2)
    
    ax.set_ylabel(gpmodel['target'])
    ax.set_xlabel(cov[var])

    plt.show()
    

# Regress out age, sex

In [ ]:
img_dat_nc_ZGP = img_dat_nc.copy()

In [ ]:
tfeat = list(img_dat_nc.loc[:,'CC_FA':'CG.R_MD'].columns.values)
tfeat.extend(img_dat_nc.loc[:, 'LLatVent':'R_cingulateC_volume'])

In [ ]:
for fff in tfeat:
    sys.stderr.write("Estimating model for " + fff + "\n" )
    tmp_mod = fitNormGP(img_dat_nc_controls, ['Age','ICV'], fff)
    tmpz    = feat2z(img_dat_nc, tmp_mod)
    img_dat_nc_ZGP.loc[:,fff] = tmpz

In [ ]:
plotGPax(img_dat_nc_ZGP, tmp_mod, 0)

In [ ]:
#Use z-scores as input into SuStaIn
img_dat_nc_ZGP.to_csv("./all_feat_GPzscores_20220715.csv")